<a href="https://colab.research.google.com/github/NH-Lee25/assignment/blob/main/junsi_sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install konlpy

In [12]:
import re
import numpy as np
import pandas as pd
import random

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf

from konlpy.tag import Kkma

In [13]:
import pandas as pd
import io
from google.colab import files
table_original = files.upload()
table = pd.read_excel(io.BytesIO(table_original['junsi_description_sudong.xlsx']))

table

Saving junsi_description_sudong.xlsx to junsi_description_sudong (1).xlsx


,Unnamed: 0,junsi_name,start,end,place,google1,Insta,Theme
0,0,환영 幻影 : 실재와 환상의 사이,2021.05.11.,2021.07.30.,갤러리 구조,"컨템포러리 아트 갤러리, 갤러리 구조가 ‘환영 幻影: 실재와 환상의 사이’ 전시를 ...",0.0,2.0
1,1,STREET NOISE,2021.02.26.,2021.06.13.,롯데월드몰,"잠실 롯데월드몰에서 2월 26일 오픈하여, SNS상에서 새로운 힙플레이스로 떠오른 ...",1.0,2.0
2,2,2020 미래작가상,2021.05.27.,2021.06.27.,캐논갤러리,수상자 김유자 문그루 최수현 의 작품전.\n\n전국 대학생을 대상으로 사진 작업을 ...,0.0,3.0
3,3,색동옷을 입은 아이들,2021.06.01.,2021.08.14.,한국색동박물관,"한국색동박물관, 대한민국역사박물관 협력 기획전 《색동옷을 입은 아이들》展일시 : 2...",0.0,0.0
4,4,나의 꽃은 가깝고 낯설다,2021.05.25.,2021.07.10.,코리아나미술관,코리아나미술관(관장 유상옥·유승희)은 ‘꽃’을 주제로 한 기획전 《나의 꽃은 가깝고...,0.0,1.0
...,...,...,...,...,...,...,...,...
370,370,김수영 : The First Village,2021.05.27.,2021.06.02.,tya갤러리,“당신의 ‘척’과 관련된 이야기를 들려주세요.”\n해방촌 tya갤러리(서울시 용산구...,0.0,2.0
371,371,조수빈 : ⫷&nbsp;⫸,2021.05.24.,2021.06.02.,공간일리,충실하게 쓸모없음을 수행하고 있는 기계들을 설명에 따라 하나하나 작동시켜 보세요. ...,0.0,2.0
372,372,이희욱 : 인간의 얼굴,2021.05.20.,2021.06.02.,에이라운지 갤러리,에이라운지는 2021년 5월 20일(목)부터 6월 2일(수)까지 이희욱 개인전 《인...,0.0,2.0
373,373,F8 소풍(逍風),2021.05.27.,2021.06.02.,아트스페이스 퀄리아,흔히 소풍(逍風)이라 하면 휴식을 취하기 위해서 야외에 나갔다 오는 일을 떠올린다....,0.0,2.0


In [14]:
# label 분류
label_0_index, label_1_index, no_label = list(), list(), list()
for idx in table.index:
    if table.loc[idx, 'Insta'] == 0:
        label_0_index.append(idx) #323
    elif table.loc[idx, 'Insta'] == 1:
        label_1_index.append(idx) #31
    else:
        no_label.append(idx) #21
print(label_0_index)
print(label_1_index)
print(no_label)
print(len(label_0_index), len(label_1_index), len(no_label))

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 93, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 135, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212, 213, 214, 216, 218, 220, 221, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 246, 248, 249, 2

In [15]:
# 단어 추출
kkma = Kkma()
words = list()
pattern = re.compile(r'\W+')

for idx, val in enumerate(table['google1']):
    print(f"\r단어 뽑아내는 중 : {idx} / {len(table['google1'])}", end='')
    
    if type(val) != str:
        words.append('')
        continue

    val = re.sub(pattern, ' ', val)
    ex_pos = kkma.pos(val)
    
    # http://kkma.snu.ac.kr/documents/?doc=postag 태그 참고. 명사만 사용
    terms = [term for term, wclass in ex_pos if wclass in ('NNG', 'NNP')]
    
    com_sep_terms = ','.join(terms)
    words.append(com_sep_terms)

print('\r------ 단어 뽑아내기 완료 ------')
table['words'] = words

------ 단어 뽑아내기 완료 ------


In [16]:
# x, y 나누기
words = table['words'].fillna('')
tfidf_vectorizer = TfidfVectorizer()
x = tfidf_vectorizer.fit_transform(words).toarray()

y = table['Insta']

In [17]:
# train, test, validation 나누기
# label_0이 label_1에 비해 너무 많음. label_0 중 랜덤 100개만 선택해서 사용, label_1은 전체를 사용

train_idx = label_0_index[:60] + label_1_index[:20] #60%
test_idx = label_0_index[60:80] + label_1_index[20:26] #20%
valid_idx = label_0_index[80:100] + label_1_index[26:] #20%

x_train, y_train = x[train_idx], y[train_idx]
x_test, y_test = x[test_idx], y[test_idx]
x_valid, y_valid = x[valid_idx], y[valid_idx]

In [18]:
# 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mse')

In [19]:
# 학습 실행
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
3/3 [==============================] - 1s 9ms/step - loss: 0.2512
Epoch 2/50
3/3 [==============================] - 0s 7ms/step - loss: 0.2443
Epoch 3/50
3/3 [==============================] - 0s 8ms/step - loss: 0.2383
Epoch 4/50
3/3 [==============================] - 0s 8ms/step - loss: 0.2307
Epoch 5/50
3/3 [==============================] - 0s 7ms/step - loss: 0.2216
Epoch 6/50
3/3 [==============================] - 0s 8ms/step - loss: 0.2110
Epoch 7/50
3/3 [==============================] - 0s 7ms/step - loss: 0.1974
Epoch 8/50
3/3 [==============================] - 0s 7ms/step - loss: 0.1825
Epoch 9/50
3/3 [==============================] - 0s 8ms/step - loss: 0.1649
Epoch 10/50
3/3 [==============================] - 0s 8ms/step - loss: 0.1464
Epoch 11/50
3/3 [==============================] - 0s 8ms/step - loss: 0.1267
Epoch 12/50
3/3 [==============================] - 0s 7ms/step - loss: 0.1066
Epoch 13/50
3/3 [==============================] - 0s 7ms/step - loss: 0.

In [20]:
# 결과 확인
y_pred = model.predict(x_valid)

for t, p in zip(y_valid, y_pred):
    print(t, '/' , p[0])

0.0 / 0.13748604
0.0 / 0.03669259
0.0 / 0.16697076
0.0 / 0.11458224
0.0 / 0.12794828
0.0 / 0.050687343
0.0 / 0.19626784
0.0 / 0.5069331
0.0 / 0.26756665
0.0 / 0.09958854
0.0 / 0.48168772
0.0 / 0.38116077
0.0 / 0.3386392
0.0 / 0.029671043
0.0 / 0.03000158
0.0 / 0.20233962
0.0 / 0.10669589
0.0 / 0.31774136
0.0 / 0.09867284
0.0 / 0.26238382
1.0 / 0.59368706
1.0 / 0.6319469
1.0 / 0.28834623
1.0 / 0.4048221
1.0 / 0.6054672
